In [3]:
import rasterio
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from rasterio.plot import show
import rasterio
import numpy as np
import requests

In [5]:
# # update the wd path to be able to laod the modules
# # os.getcwd()
os.chdir('..')
os.getcwd()

'C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\cities-indicators-framework\\citymetrix\\cities-cif'

In [3]:
os.environ['GCS_BUCKET']='gee-exports'
os.environ['GOOGLE_APPLICATION_USER']='developers@citiesindicators.iam.gserviceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='C:\\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities-indicators-framework\citymetrix\credentials-citiesindicators.json'

In [6]:
os.environ['GCS_BUCKET']='gee-exports'
os.environ['GOOGLE_APPLICATION_USER']='developers@citiesindicators.iam.gserviceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='C:\\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities-indicators-framework\citymetrix\credentials-citiesindicators.json'

In [7]:
from city_metrix import (
    built_land_without_tree_cover, 
    mean_tree_cover, 
    built_land_with_low_surface_reflectivity, 
    built_land_with_high_land_surface_temperature
)


Authenticating to GEE with configured credentials file.


# Select a city

In [8]:
boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4union.geojson'
city_gdf = gpd.read_file(boundary_path, driver='GeoJSON')
city_gdf.head()


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."


# Calculate indicators - AOI level

## Compute indicators: One city * One indicator

### HEA-3 Built land with Low Surface reflectivity

In [7]:
%%time

city_gdf["built_land_with_low_surface_reflectivity"] = built_land_with_low_surface_reflectivity(city_gdf) 
city_gdf

Extracting ESA world cover layer:
[########################################] | 100% Completed | 15.2s


NameError: name 'ProgressBar' is not defined

### LND-2 Average Tree cover

In [9]:
%%time

city_gdf["mean_tree_cover"] = mean_tree_cover(city_gdf) 
city_gdf

Extracting tree cover layer:
[########################################] | 100% Completed | 27.5s
CPU times: total: 10.4 s
Wall time: 31 s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366773


### HEA-4 Percent of Built land without tree cover

In [10]:
%%time

city_gdf["built_land_without_tree_cover"] = built_land_without_tree_cover(city_gdf) 
city_gdf

Extracting ESA world cover layer:
[########################################] | 100% Completed | 16.0s
Extracting tree cover layer:
[########################################] | 100% Completed | 29.4s
Extracting ESA world cover layer:
[########################################] | 100% Completed | 16.0s
CPU times: total: 25.7 s
Wall time: 1min 8s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover,built_land_without_tree_cover
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366773,0.907382


### HEA-2 Built Land With High LST

In [11]:
%%time

city_gdf["built_land_with_high_land_surface_temperature"] = built_land_with_high_land_surface_temperature(city_gdf) 
city_gdf

Extracting ESA world cover layer:
[########################################] | 100% Completed | 13.8s


TypeError: unhashable type: 'numpy.ndarray'

## Compute indicators: One city * Multiple indicators

In [40]:
%%time

indicators =get_indicators(city_gdf, indicators=[Indicator.TREE_COVER,
                                                 Indicator.BUILT_LAND_WITH_TREE_COVER])
indicators_df = pd.merge(indicators[0], indicators[1])
indicators_df

CPU times: total: 3.31 s
Wall time: 8.63 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,LND_2_percentTreeCover,HEA_4_percentBuiltupWithoutTreeCover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.283478,0.907352


## Multiple cities

In [19]:
# Calculate indicator for multiple regions of interest

boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4union.geojson'
city_Salvador = gpd.read_file(boundary_path, driver='GeoJSON')

boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-IDN-Jakarta-ADM4union.geojson'
city_Jakarta = gpd.read_file(boundary_path, driver='GeoJSON')

cities = pd.concat([city_Salvador, city_Jakarta])
cities = cities.reset_index(drop=True)
cities

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."
1,IDN-Jakarta_ADM-4-union_1,ADM-4-union,IDN-Jakarta,IDN-Jakarta,2022-06-27,"MULTIPOLYGON (((106.78141 -6.31616, 106.78124 ..."


In [20]:
# loop over cities
for i in range(0,len(cities)):
    city = cities.loc[[i]]
    city_indicator = float(mean_tree_cover(city))
    cities.loc[i,"mean_tree_cover"] = city_indicator

Extracting tree cover layer:
[########################################] | 100% Completed | 26.7s
Extracting tree cover layer:
[########################################] | 100% Completed | 22.3s


In [21]:
cities

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366773
1,IDN-Jakarta_ADM-4-union_1,ADM-4-union,IDN-Jakarta,IDN-Jakarta,2022-06-27,"MULTIPOLYGON (((106.78141 -6.31616, 106.78124 ...",15.871722


# Get layers

In [17]:
from city_metrix.layers import EsaWorldCoverClass, EsaWorldCover, TreeCover, HighLandSurfaceTemperature, LandSurfaceTemperature, Albedo

## Esa World Cover

In [9]:
# Load World cover layer
city_LandCover = EsaWorldCover().get_data(city_gdf.total_bounds)
city_LandCover

Extracting ESA world cover layer:
[########################################] | 100% Completed | 22.3s


<xarray.DataArray 'Map' (y: 2878, x: 3722)>
array([[80., 80., 80., ..., 30., 30., 30.],
       [80., 80., 80., ..., 30., 30., 30.],
       [80., 80., 80., ..., 30., 30., 30.],
       ...,
       [50., 50., 50., ..., 80., 80., 80.],
       [50., 50., 50., ..., 80., 80., 80.],
       [50., 50., 50., ..., 80., 80., 80.]])
Coordinates:
    time     datetime64[ns] 2020-01-01
  * x        (x) float32 -38.65 -38.65 -38.65 -38.65 ... -38.3 -38.3 -38.3 -38.3
  * y        (y) float32 -12.76 -12.76 -12.76 -12.76 ... -13.02 -13.02 -13.02
Attributes:
    id:             Map
    data_type:      {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max'...
    dimensions:     [4320000, 1728000]
    crs:            EPSG:4326
    crs_transform:  [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-0...

## Trpical Tree cover

In [8]:
# Load layer
city_TreeCover = TreeCover().get_data(city_gdf.total_bounds)
city_TreeCover

Extracting tree cover layer:
[########################################] | 100% Completed | 23.0s


<xarray.DataArray 'ttc' (y: 2878, x: 3722)>
array([[ nan,  nan,  nan, ..., 100.,  90.,  90.],
       [ nan,  nan,  nan, ...,  90.,  90.,  90.],
       [ nan,  nan,  nan, ...,  90.,  90.,  80.],
       ...,
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])
Coordinates:
    time     int32 0
  * x        (x) float32 -38.65 -38.65 -38.65 -38.65 ... -38.3 -38.3 -38.3 -38.3
  * y        (y) float32 -12.76 -12.76 -12.76 -12.76 ... -13.02 -13.02 -13.02
Attributes:
    id:             ttc
    data_type:      {'type': 'PixelType', 'precision': 'double', 'min': 0, 'm...
    crs:            EPSG:4326
    crs_transform:  [1, 0, 0, 0, 1, 0]

## LST

In [11]:
city_high_lst = HighLandSurfaceTemperature().get_data(city_gdf.total_bounds)
city_high_lst

TypeError: unhashable type: 'numpy.ndarray'

In [14]:
city_lst = LandSurfaceTemperature(start_date="2013-01-01", end_date="2023-01-01").get_data(city_gdf.total_bounds)
city_lst

TypeError: unhashable type: 'numpy.ndarray'

## ALbedo

In [15]:
# load boundary of sub-georgaphy
boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4.geojson'
city_gdf_sub = gpd.read_file(boundary_path, driver='GeoJSON')
city_gdf_sub.head()

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.40125 -12.96457, -38.40126..."
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.39898 -12.96269, -38.39898..."
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.37346 -12.93345, -38.37386..."
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.41741 -12.97578, -38.41746..."
4,BRA-Salvador_ADM4_5,ADM4,Jardim Armação,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.43383 -12.98742, -38.43386..."


In [16]:
city_gdf_sub = city_gdf_sub.head(1)
city_gdf_sub

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.40125 -12.96457, -38.40126..."


In [18]:
city_albedo = Albedo().get_data(city_gdf_sub.total_bounds)
city_albedo

NameError: name 'ProgressBar' is not defined